In [1]:
!pip install git+https://github.com/sberbank-ai/Real-ESRGAN.git

  Cloning https://github.com/sberbank-ai/Real-ESRGAN.git to /tmp/pip-req-build-nwsjnba3
  Running command git clone --filter=blob:none --quiet https://github.com/sberbank-ai/Real-ESRGAN.git /tmp/pip-req-build-nwsjnba3
  Resolved https://github.com/sberbank-ai/Real-ESRGAN.git to commit 362a0316878f41dbdfbb23657b450c3353de5acf
  Preparing metadata (setup.py) ... done
  Created wheel for RealESRGAN: filename=RealESRGAN-1.0-py3-none-any.whl size=9105 sha256=a56ebedd81378bf97979f1ccc6bb13b3b14d5930301370231f2c9cdcfec2a29d
  Stored in directory: /tmp/pip-ephem-wheel-cache-awcbyd5t/wheels/dd/a3/7d/774508ed192455403be294c8a6c4ad1c83dde8fcdd8903e64a
Successfully built RealESRGAN


In [2]:
from RealESRGAN import RealESRGAN
from PIL import Image
import numpy as np
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)

model_scale = "4" #@param ["2", "4", "8"] {allow-input: false}

model = RealESRGAN(device, scale=int(model_scale))
model.load_weights(f'weights/RealESRGAN_x{model_scale}.pth')

device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:672: FutureWarning: 'cached_download' is the legacy way to download files from the HF hub, please consider upgrading to 'hf_hub_download'
  warnings.warn(


RealESRGAN_x4.pth:   0%|          | 0.00/67.0M [00:00<?, ?B/s]

Weights downloaded to: weights/RealESRGAN_x4.pth


In [3]:
!pip install lpips
# take denoised vs noisy sr gan input

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 1.4 MB/s eta 0:00:00


In [3]:
import cv2
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np

image = cv2.imread("/content/noise_inputs/window.jpg")

# metrics so I can test if img is noisy
skewness1 = np.mean((image - np.mean(image))**3) / np.mean((image - np.mean(image))**2)**(3/2)
kurtosis1 = np.mean((image - np.mean(image))**4) / np.mean((image - np.mean(image))**2)**2
print(skewness1, kurtosis1)
if abs(skewness1) < 1.0 or abs(kurtosis1) < 3:
  print("Denoising")
# Apply non-local means denoising to the color image
  denoised = cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 21)
else:
  denoised = image
i = 17
cv2.imwrite("denoised_image.jpg", denoised)

with Image.open('denoised_image.jpg') as img:
  # img.save(f'/content/og_inputs/result_{i}.jpg')
  resized_img = img.resize((500, 667))
  resized_img.save(f'/content/inputs/resized_result_{i}.jpg')
psnr_value = cv2.PSNR(image, denoised)

# plt.subplot(1, 2, 1), plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)), plt.title('Original Image')
# plt.xticks([]), plt.yticks([])
# plt.subplot(1, 2, 2), plt.imshow(cv2.cvtColor(denoised, cv2.COLOR_BGR2RGB)), plt.title('Denoised Image')
# plt.xticks([]), plt.yticks([])
# plt.show()
print("PSNR NL means:",psnr_value)

0.6583283502937303 2.891414679754546
Denoising
PSNR NL means: 28.645152599943593


In [14]:
import lpips
import torch
from PIL import Image
import numpy as np
# Load your images
image2 = Image.open("/content/og_inputs/result_23.jpg")
image1 = Image.open("/content/noise_inputs/23_LL.jpg")
image1 = np.array(image1)
image2 = np.array(image2)
# Convert images to tensors
image1_tensor = lpips.im2tensor(image1).cuda()
image2_tensor = lpips.im2tensor(image2).cuda()

# Create a model (you can choose different versions, e.g., 'alex', 'vgg', 'squeeze', 'mobile')
loss_fn_alex = lpips.LPIPS(net='squeeze', version='0.1')
loss_fn_alex.cuda()  # Use GPU for faster computation if available

# Compute the LPIPS score
lpips_score = loss_fn_alex(image1_tensor, image2_tensor)

print(f"LPIPS Score: {lpips_score.item()}")


Setting up [LPIPS] perceptual loss: trunk [squeeze], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.10/dist-packages/lpips/weights/v0.1/squeeze.pth


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_1_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


OutOfMemoryError: ignored

In [5]:
import os
from google.colab import files
import shutil
from io import BytesIO
import io
import tarfile

upload_folder = 'inputs'
result_folder = 'results'

os.makedirs(upload_folder, exist_ok=True)
os.makedirs(result_folder, exist_ok=True)

IMAGE_FORMATS = ('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')

def image_to_tar_format(img, image_name):
    buff = BytesIO()
    if '.png' in image_name.lower():
        img = img.convert('RGBA')
        img.save(buff, format='PNG')
    else:
        img.save(buff, format='JPEG')
    buff.seek(0)
    fp = io.BufferedReader(buff)
    img_tar_info = tarfile.TarInfo(name=image_name)
    img_tar_info.size = len(buff.getvalue())
    return img_tar_info, fp

def process_tar(path_to_tar):
    processing_tar = tarfile.open(path_to_tar, mode='r')
    result_tar_path = os.path.join('results/', os.path.basename(path_to_tar))
    save_tar = tarfile.open(result_tar_path, 'w')

    for c, member in enumerate(processing_tar):
        print(f'{c}, processing {member.name}')

        if not member.name.endswith(IMAGE_FORMATS):
            continue

        try:
            img_bytes = BytesIO(processing_tar.extractfile(member.name).read())
            img_lr = Image.open(img_bytes, mode='r').convert('RGB')
        except Exception as err:
            print(f'Unable to open file {member.name}, skipping')
            continue

        img_sr = model.predict(np.array(img_lr))
        # adding to save_tar
        img_tar_info, fp = image_to_tar_format(img_sr, member.name)
        save_tar.addfile(img_tar_info, fp)

    processing_tar.close()
    save_tar.close()
    print(f'Finished! Archive saved to {result_tar_path}')

def process_input(filename):
    if tarfile.is_tarfile(filename):
        process_tar(filename)
    else:
        result_image_path = os.path.join('results/', os.path.basename(filename))
        image = Image.open(filename).convert('RGB')
        sr_image = model.predict(np.array(image))
        # sr_image.resize((4000,4000))
        sr_image.show()
        sr_image.save(result_image_path)
        print(f'Finished! Image saved to {result_image_path}')

# upload files
# uploaded = files.upload()
# for filename in uploaded.keys():
inpts = [f for f in os.listdir('inputs')]
# print(inputs)
for filename in inpts:
    print('Processing:', filename)
    newfilename = '/content/inputs/'+filename
    process_input(newfilename)

Processing: log_resized.jpeg
Finished! Image saved to results/log_resized.jpeg
Processing: .ipynb_checkpoints


IsADirectoryError: ignored

In [ ]:
# Load your images
image1 = Image.open("path/to/image1.jpg")
image2 = Image.open("path/to/image2.jpg")

# Convert images to tensors
image1_tensor = lpips.im2tensor(image1)
image2_tensor = lpips.im2tensor(image2)

# Create a model (you can choose different versions, e.g., 'alex', 'vgg', 'squeeze', 'mobile')
loss_fn_alex = lpips.LPIPS(net='alex', version='0.1')
loss_fn_alex.cuda()  # Use GPU for faster computation if available

# Compute the LPIPS score
lpips_score = loss_fn_alex(image1_tensor, image2_tensor)

print(f"LPIPS Score: {lpips_score.item()}")
